## Import modules

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from dataset_exploration.utils import create_histogram, read_annotation
from configs.utils import get_dataset_dir
import sys
from analyze_stuff import get_config, get_dataloader
from vizer.draw import draw_boxes
from ssd import utils
from tqdm import tqdm
import cv2
import numpy as np

/home/z/pro/Computer_Vision_Project/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

# Using pandas

## Image analyzer

In [2]:
def convert_boxes_coords_to_pixel_coords(boxes, width, height):
    boxes_for_first_image = boxes[0]  # This is the only image in batch
    boxes_for_first_image[:, [0, 2]] *= width
    boxes_for_first_image[:, [1, 3]] *= height
    return boxes_for_first_image.cpu().numpy()


def convert_image_to_hwc_byte(image, rgb):
    first_image_in_batch = image[0].clone().detach()  # This is the only image in batch
    if rgb != None:
        if rgb == 0:
            first_image_in_batch[1] = first_image_in_batch[rgb]
            first_image_in_batch[2] = first_image_in_batch[rgb]
        if rgb == 1:
            first_image_in_batch[0] = first_image_in_batch[rgb]
            first_image_in_batch[2] = first_image_in_batch[rgb]
        if rgb == 2:
            first_image_in_batch[1] = first_image_in_batch[rgb]
            first_image_in_batch[0] = first_image_in_batch[rgb]
    image_pixel_values = (first_image_in_batch * 255).byte()
    image_h_w_c_format = image_pixel_values.permute(1, 2, 0)

    return image_h_w_c_format.cpu().numpy()


def visualize_boxes_on_image(batch, label_map, rgb):
    image = convert_image_to_hwc_byte(batch["image"], rgb)
    boxes = convert_boxes_coords_to_pixel_coords(batch["boxes"], batch["width"], batch["height"])
    labels = batch["labels"][0].cpu().numpy().tolist()

    image_with_boxes = draw_boxes(image, boxes, labels, class_name_map=label_map)
    return image_with_boxes


def create_viz_image(batch, label_map):
    image = visualize_boxes_on_image(batch, label_map, None)
    image_r = visualize_boxes_on_image(batch, label_map, 0)
    image_g = visualize_boxes_on_image(batch, label_map, 1)
    image_b = visualize_boxes_on_image(batch, label_map, 2)

    # We concatinate in the height axis, so that the images are placed on top of
    # each other
    concatinated_image = np.concatenate([
        image,
        image_b,
        image_r,
        image_g,
    ], axis=0)
    return concatinated_image


def create_filepath(save_folder, image_id):
    filename = "image_" + str(image_id) + ".png"
    return os.path.join(save_folder, filename)


def save_images(dataloader, cfg, save_folder, num_images_to_visualize):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    print("Saving images to", save_folder)

    num_images_to_save = min(len(dataloader), num_images_to_visualize)
    dataloader = iter(dataloader)

    for i in tqdm(range(num_images_to_save)):
        batch = next(dataloader)
        viz_image = create_viz_image(batch, cfg.label_map)
        filepath = create_filepath(save_folder, i)
        # viz_image = cv2.cvtColor(viz_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(filepath, viz_image)

In [3]:
config_path = "../configs/tdt4265.py"
cfg = get_config(config_path)
dataset_to_analyze = "train"  # or "val"
dataloader = get_dataloader(cfg, dataset_to_analyze)

num_images_to_visualize = 200
save_folder = os.path.join("red_images")
save_images(dataloader, cfg, save_folder, num_images_to_visualize)


Saving images to red_images


100%|██████████| 200/200 [01:36<00:00,  2.08it/s]
